<a href="https://colab.research.google.com/github/lesanpi/Clasificacion-de-genero-por-nombres./blob/master/Reto_Clasificacion_Nombres_lesanpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importamos Pandas, Numpy, NLTK y Random**

**Tambien hacer un git clone del repositorio en github.**


In [14]:
import pandas as pd
import numpy as np
import nltk, random
!git clone https://github.com/jvalhondo/spanish-names-surnames

fatal: destination path 'spanish-names-surnames' already exists and is not an empty directory.


# **Cargamos la Data.**

Si vemos en google colab se cargaron diferentes archivos .csv que contienen los nombres masculinos y femeninos por separado.

Lo que hice, fue crear una funcion que dada el path del archivo, leyera, sacara los nombres y segun el genero que le indicaramos creara un array con el (nombre, genero).

Se unio los datos de nombres masculinos y femeninos y se hizo un shuffle para que no quedaran ordenados por genero.


In [15]:
def fromCSVtoArray(dataPath, genre):
  x_names = pd.read_csv(dataPath)
  x_names = x_names.dropna(axis = 0)
  x_names = x_names['name']
  x_names = np.asarray(x_names)

  return [(name, genre) for name in x_names] 

female = fromCSVtoArray('/content/spanish-names-surnames/female_names.csv', 'female')
male = fromCSVtoArray('/content/spanish-names-surnames/male_names.csv', 'male')
data = female + male

random.shuffle(data)
data[:10]

[('YANIER', 'male'),
 ('XIANZHONG', 'male'),
 ('CRISTO RAMON', 'male'),
 ('SILVIA AURORA', 'female'),
 ('MARIA FRUCTUOSA', 'female'),
 ('ROSANNA MARIA', 'female'),
 ('DJAMEL EDDINE', 'male'),
 ('ROMAN JAVIER', 'male'),
 ('MUHAMMAD', 'male'),
 ('DEIMANTE', 'female')]

Podemos guardar el array de nuestros datos, de tal forma siempre trabajaremos con el mismo dataset de entrenamiento y de test cuando se separe.

In [22]:
np.save('nombres.npy', data)

In [24]:
data = np.load('nombres.npy')

# **Indicaremos las funciones que utilizamos para obtener los atributos**


* ultimaLetra: Solo utiliza la ultima letra.
* atributos: Primera y ultima letra.
* atributos1: Primer y ultima letra, conteo de cuantas letras, y si contiene o no una letra.
* atributos2: Separa los nombres con split(' '), para trabajar con el segundo nombre en caso de que lo tenga. Aplica las mismas funciones que atributos1 pero almacena tanto del segundo como el primer nombre. 
* atributos3: Utiliza la misma logica que atributos2, solo que en vez de analizar todas las letras del abecedario, recorre solamente las vocales.

In [16]:
def ultimaLetra(nombre):
    atrib = {}
    atrib["ultima_letra"] = nombre[-1].lower()
    
    return atrib

In [17]:
def atributos(nombre):
    atrib = {}
    atrib["primera_letra"] = nombre[0].lower()
    atrib["ultima_letra"] = nombre[-1].lower()
    
    return atrib

In [18]:
def atributos1(nombre):
    atrib = {}
    atrib["primera_letra"] = nombre[0].lower()
    atrib["ultima_letra"] = nombre[-1].lower()
    for letra in 'abcdefghijklmnopqrstuvwxyz':
        atrib["count({})".format(letra)] = nombre.lower().count(letra)
        atrib["has({})".format(letra)] = (letra in nombre.lower())
    return atrib

In [19]:
def atributos2(nombre):
    primerSegundoNombre = nombre.split(' ')

    atrib = {}
    for i in range(len(primerSegundoNombre)):
      atrib["primera_letra_{}".format(i + 1)] = primerSegundoNombre[i][0].lower()
      atrib["ultima_letra_{}".format(i + 1)] = primerSegundoNombre[i][-1].lower()
      for letra in 'abcdefghijklmnopqrstuvwxyz':
          atrib["count({})_{}".format(letra, i + 1)] = primerSegundoNombre[i].lower().count(letra)
          atrib["has({})_{}".format(letra, i + 1)] = (letra in primerSegundoNombre[i].lower())
    return atrib

In [35]:
def atributos3(nombre):
    primerSegundoNombre = nombre.split(' ')

    atrib = {}
    for i in range(len(primerSegundoNombre)):
      atrib["primera_letra_{}".format(i + 1)] = primerSegundoNombre[i][0].lower()
      atrib["ultima_letra_{}".format(i + 1)] = primerSegundoNombre[i][-1].lower()
      for letra in 'aeiou':
          atrib["count({})_{}".format(letra, i + 1)] = primerSegundoNombre[i].lower().count(letra)
          atrib["has({})_{}".format(letra, i + 1)] = (letra in primerSegundoNombre[i].lower())
    return atrib

# **Fase de entrenamiento del Clasificador NaiveBayes**

Utilizamos la funcion para preparar la data y luego entrenar el clasificar con la data.

In [38]:
fset = [(atributos3(n), g) for (n, g) in data]
train, test = fset[500:], fset[:500]
fset
classifierEspaniol = nltk.NaiveBayesClassifier.train(train)

# **Accuracy de nuestro modelo.**

Utilizando el mismo dataset, podemos ver que los resultados de accuracy que nos dio con distintos atributos:

* ultimaLetra: 0.792
* atributos: 0.8
* atributos1: 0.836
* atributos2: 0.858
* **atributos3: 0.858**

Podriamos decir que la mejor funcion de atributos fue la de **atributos3**, ya que no solo recorrio las vocales y no todas las letras del abecedario y tuvo el mismo performance. 

**Nota:** Se podria intentar ver que otra informacion obtener del nombre, y ver si es mejor recorrer todo el abecedario o solo las vocales. 

In [39]:
print(nltk.classify.accuracy(classifierEspaniol, test))

0.858
